# Census Data Tools

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

In [ ]:
import morpc

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [ ]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [ ]:
api = morpc.census.api_get(url, params)

In [ ]:
api

## American Community Survey (ACS) Data Class

When using ACS data, generally we will be digesting data produded using the [morpc-censusacs-fetch](https://github.com/morpc/morpc-censusacs-fetch) workflow. The data that is produced from that script is by default saved in its output_data folders ./morpc-censusacs-fetch/output_data/

The Census ACS Fetch script leverages the `acs_data` class form `morpc.census`


### Create an initial object which represents a variable in the ACS data api.

The class takes 3 arguments:

1. variable group number
2. the year
3. the type of survey (1 or 5 year estimates)

In [ ]:
from morpc.census import acs_data

In [ ]:
acs = acs_data('B01001', '2023', '5')

The initial call creates queries the Census for the variable definitions and returns a dictionary of the available variables in the group. see `acs.VARS`

In [ ]:
acs.VARS

The initial call alse fetchs a list of dimensions from a cached json file in ./morpc/census/acs_variable_group.json and is stored in morpc.census.ACS_VAR_GROUPS.

#### Manual verfication for variable dimension names. 

The list of dimensions are automatically created from the Census Variable labels and need verified before being used. If the dimesion names have not be verified, the will not be stored. Navigate to the JSON and check to make sure that there are the correct number of dimension and that they are in the correct order. Change the verfication field to `true`.

In [ ]:
acs.DIMENSIONS

### Query the API for the deisred variables and geography

The `.query()` method queries the API and caches the data in memory under `acs.DATA`. At the same time it creates a frictionless schema that corrosponds with the data. 

#### scope:
These are pre-defined sumlevels and scopes for commonly queried geographies. see `morpc.census.SCOPES`.

In [ ]:
acs = acs.query(for_param='county:*', in_param='state:39')

### For custom queries, use for and in parameters to pass to api query. 

#### for_param:
(optional) The geographies for which to call the the query "state:*" represents all states. "state:39" represent Ohio.

#### in_param:
(optional) A filter for the for parameter. In combinations this allows you do call for small geograhpies inside larger ones. 

> Examples: for_param="county:\*", in_param="state:39" would get all counties in Ohio.
> for_param="tract:\*", in_param='state:39,county:041,049' gets all census tracts in Delaware and Franklin Counties.

### Filter the variables using the get parameter

#### get_param:
(Optional) If you want to return a subset of variables, they can be passed here as a list.

### Dimension Tables

When the query is called the class makes table with the dimensions included that can be used to get summaries of the data. 

This can be used to get quick queries for summaries. 

In [ ]:
acs.DIM_TABLE.LONG

In [ ]:
acs.DIM_TABLE.WIDE

In [ ]:
acs.DIM_TABLE.PERCENT

### Save raw data (not dim table) as a frictionless resource with schema

After querying the data, save the data as a frictionless resource with reasonable descriptors. 

In [ ]:
acs.save(output_dir='./temp_data/')

In [ ]:
acs.SCHEMA

In [ ]:
acs.RESOURCE

## Load data from cached file

In [ ]:
import morpc

In [ ]:
acs = morpc.census.ACS('B25010', '2023', '5').load(scope='region15-tracts', dirname='./temp_data/')

## Georeference the data to map

Add geometries by joining GEOS to DATA.

In [ ]:
acs.GEOS

In [ ]:
import geopandas as gpd
acs.DATA = gpd.GeoDataFrame(acs.DATA.join(acs.GEOS), geometry='geometry')

In [ ]:
acs.DATA.plot(column='B01001_002E')

## Use the built in .explore() method to view a map of all the columns in data

In [ ]:
acs.explore(table='PERCENTS')

In [ ]:
acs.MAP

## Using the rest_api module to fetch geometry data from Census API

In [ ]:
import morpc.rest_api as rest_api
import morpc.census as census

In [ ]:

url =  rest_api.get_layer_url(2024, 'county subdivisions', survey='ACS')

query = "STATE = '39' and COUNTY = '049'"

resource = rest_api.resource(
    name = 'morpc-franklin-tracts',
    url = url,
    where = query,
    max_record_count=500
)

In [ ]:
gdf = rest_api.gdf_from_resource(resource)

## Below should still be functional, but hoping to implement into ACS class

#### Load the data using frictionless.load_data()

In [ ]:
data, resource, schema = morpc.frictionless.load_data('./temp_data/morpc-acs5-2023-state-B01001.resource.yaml', verbose=False)

#### Using ACS_ID_FIELDS to get the fields ids

In [ ]:
morpc.census.acs_generate_universe_table(data.set_index("GEO_ID"), "B01001_001")

#### Create a dimension table with the data and the dimension names

In [ ]:
dim_table = morpc.census.acs_generate_dimension_table(data.set_index("GEO_ID"), schema, idFields=idFields, dimensionNames=["Sex", "Age group"])

In [ ]:
dim_table.loc[dim_table['Variable type'] == 'Estimate'].head()

### Build ACS Variable Group JSON for Dimension names

In [ ]:
# import requests
# import pandas as pd
# from numpy import nan
# from IPython.display import clear_output

# r = requests.get('https://api.census.gov/data/2023/acs/acs5/variables.json')
# varjson = r.json()

# groups = {}
# for variable in varjson['variables']:
#     if variable not in ['for', 'in', 'ucgid', 'GEO_ID', 'AIANHH', 'AIHHTL', 'AIRES', 'ANRC']:
#         group = varjson['variables'][variable]['group']
#         if not group[-1].isalpha():
#             if group not in groups:
#                 groups[group] = {}
#                 groups[group]['concept'] = varjson['variables'][variable]['concept']
#                 variables = {}
#                 for variable in varjson['variables']:
#                     if varjson['variables'][variable]['group'] == group:
#                         variables[variable] = varjson['variables'][variable]['label'].replace(":","").split('Estimate!!')[-1].split('!!')
#                 variables = {k: v for k, v in sorted(variables.items(), key=lambda item: item[0])}
#                 groups[group]['variables'] = variables
# groups = {k: v for k, v in sorted(groups.items(), key=lambda item: item[0])}
# for group in groups:
#     print(group)
#     clear_output(wait=True)
#     variables = groups[group]['variables']
#     var_list = [v for k, v in variables.items()]
#     var_df = pd.DataFrame(var_list)
#     var_set = set([item for sublist in var_list for item in sublist])
#     var_set.discard('')
#     var_columns = {}
#     for var in var_set:
#         var_columns[var] = {}
#         for column in var_df.columns:
#             if var in var_df[column].value_counts():
#                 count = var_df[column].value_counts()[var]
#                 var_columns[var][column] = count
#     column_map = {}
#     for column in var_columns:
#         column_map[column] = max(var_columns[column], key=var_columns[column].get)
#     for k, v in column_map.items():
#         for column in var_df.columns:
#             for i, row in var_df.iterrows():
#                 if var_df.iloc[i, column] == k:
#                     var_df.iloc[i, column] = None
#                     var_df.iloc[i, v] = k
#                     var_df = var_df.replace(nan, '-')
#     var_dict = {}
#     for i, row in var_df.iterrows():
#         var_dict[[x for x in variables.keys()][i]] = row.to_list()
#     groups[group] = var_dict

In [ ]:
# import json
# with open('../morpc/census/acs_variable_groups_v2.json', 'w') as file:
#     json.dump(groups, file, indent=2)

In [1]:
import morpc

In [4]:
acs = morpc.census.ACS('B02001', 2023, 5).query(for_param='county:*', in_param='state:39')

MESSAGE | morpc.census.ACS.init | Initializing ACS object for B02001 for 2023 ACS 5-year survey ...
MESSAGE | morpc.census.ACS.init | Loading variable dictionary for B02001 for 5-year survey in 2023...
MESSAGE | morpc.census.ACS.query | Querying data for B02001 for 5-year survey in 2023...
MESSAGE | morpc.census.ACS.query | NAME set to morpc-acs5-2023-custom-b02001-20250827...
MESSAGE | morpc.census.ACS.define_schema | Defining schema for B02001 for 5-year survey in 2023...
MESSAGE | morpc.census.ACS.define_schema | Schema is valid.
MESSAGE | morpc.census.ACS.query | Querying data from https://api.census.gov/data/2023/acs/acs5 with parameters: /n/n {'get': 'GEO_ID,NAME,B02001_001E,B02001_001M,B02001_002E,B02001_002M,B02001_003E,B02001_003M,B02001_004E,B02001_004M,B02001_005E,B02001_005M,B02001_006E,B02001_006M,B02001_007E,B02001_007M,B02001_008E,B02001_008M,B02001_009E,B02001_009M,B02001_010E,B02001_010M', 'for': 'county:*', 'in': 'state:39'}...
Total variables requested: 22
Starting r

In [5]:
acs.map(table='PERCENTS', verbose=True)

MESSAGE | morpc.census.ACS.explore | Joining geometries to data...
MESSAGE | morpc.census.ACS.explore | Creating map...
morpc.plot.map.MAP: Processing column 1 of 10 : Total, American Indian and Alaska Native alone
morpc.plot.map.MAP: Processing column 2 of 10 : Total, Asian alone
morpc.plot.map.MAP: Processing column 3 of 10 : Total, Black or African American alone
morpc.plot.map.MAP: Processing column 4 of 10 : Total, Native Hawaiian and Other Pacific Islander alone
morpc.plot.map.MAP: Processing column 5 of 10 : Total, Some Other Race alone
morpc.plot.map.MAP: Processing column 6 of 10 : Total, Two or More Races
morpc.plot.map.MAP: Processing column 7 of 10 : Total, Two or More Races, Two races excluding Some Other Race, and three or more races
morpc.plot.map.MAP: Processing column 8 of 10 : Total, Two or More Races, Two races including Some Other Race
morpc.plot.map.MAP: Processing column 9 of 10 : Total, White alone
morpc.plot.map.MAP: Processing column 10 of 10 : geometry
morpc.p